Market depth (order book)
==============

In [1]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=16)

To get a list of all exchanges that support market depth data and display the first five:

In [2]:
l = ib.reqMktDepthExchanges()
l[:5]

[DepthMktDataDescription(exchange='DTB', secType='OPT', serviceDataType='Deep'),
 DepthMktDataDescription(exchange='LSEETF', secType='STK', serviceDataType='Deep'),
 DepthMktDataDescription(exchange='SGX', secType='FUT', serviceDataType='Deep'),
 DepthMktDataDescription(exchange='IDEALPRO', secType='CASH', serviceDataType='Deep', aggGroup=4),
 DepthMktDataDescription(exchange='ARCA', secType='STK', serviceDataType='Deep')]

Let's subscribe to market depth data for EURUSD:

In [3]:
eurusd = Forex('EURUSD')
ticker = ib.reqMktDepth(eurusd)

To see the order book live updated:

In [4]:
from IPython.display import clear_output
import pandas as pd
import time
runUntil = time.time() + 15

df = pd.DataFrame(index=range(5),
        columns='bidSize bidPrice askPrice askSize'.split())

while ib.waitOnUpdate():
    l = ticker.domBids
    for i in range(5):
        df.iloc[i, 0] = l[i].size if i < len(l) else 0
        df.iloc[i, 1] = l[i].price if i < len(l) else 0
    l = ticker.domAsks
    for i in range(5):
        df.iloc[i, 2] = l[i].price if i < len(l) else 0
        df.iloc[i, 3] = l[i].size if i < len(l) else 0
    clear_output(wait=True)
    display(df)
    if time.time() > runUntil:
        break

,bidSize,bidPrice,askPrice,askSize
0,20000000,1.1796,1.17965,1000000
1,1000000,1.17955,1.1797,21080000
2,7000000,1.1795,1.17975,10000000
3,1000000,1.17945,1.1798,1000000
4,50000,1.1793,0,0


Stop the market depth subscription:

In [5]:
ib.cancelMktDepth(eurusd)

In [6]:
ib.disconnect()